## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,San Policarpo,PH,12.1791,125.5072,83.30,74,98,11.68,overcast clouds
1,1,Bredasdorp,ZA,-34.5322,20.0403,61.36,81,6,8.52,clear sky
2,2,Butaritari,KI,3.0707,172.7902,81.39,73,75,14.61,broken clouds
3,3,Hot Springs,US,34.5037,-93.0552,72.00,23,0,4.61,clear sky
4,4,Pochutla,MX,15.7432,-96.4661,81.82,77,100,5.10,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
#3.a-2 input statemenst to prompt usuer to enter their min and max temp critieria for their vacation.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 84


In [5]:
#3.b filter data to find the cities use loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,San Policarpo,PH,12.1791,125.5072,83.30,74,98,11.68,overcast clouds
2,2,Butaritari,KI,3.0707,172.7902,81.39,73,75,14.61,broken clouds
4,4,Pochutla,MX,15.7432,-96.4661,81.82,77,100,5.10,overcast clouds
11,11,Padang,ID,-0.9492,100.3543,75.25,72,5,3.44,clear sky
20,20,Poum,NC,-20.2333,164.0167,81.16,69,49,7.99,scattered clouds
27,27,Bengkulu,ID,-3.8004,102.2655,76.66,79,64,5.95,broken clouds
31,31,Hithadhoo,MV,-0.6000,73.0833,82.40,69,9,8.97,clear sky
32,32,Thongwa,MM,16.7619,96.5278,78.96,74,97,3.60,overcast clouds
33,33,Hilo,US,19.7297,-155.0900,80.33,73,75,13.80,broken clouds
37,37,Lucea,JM,18.4510,-78.1736,80.46,70,98,9.33,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                164
City                   164
Country                164
Lat                    164
Lng                    164
Max Temp               164
Humidity               164
Cloudiness             164
Wind Speed             164
Current Description    164
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_cities_df =preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,San Policarpo,PH,83.30,overcast clouds,12.1791,125.5072,
2,Butaritari,KI,81.39,broken clouds,3.0707,172.7902,
4,Pochutla,MX,81.82,overcast clouds,15.7432,-96.4661,
11,Padang,ID,75.25,clear sky,-0.9492,100.3543,
20,Poum,NC,81.16,scattered clouds,-20.2333,164.0167,
27,Bengkulu,ID,76.66,broken clouds,-3.8004,102.2655,
31,Hithadhoo,MV,82.40,clear sky,-0.6000,73.0833,
32,Thongwa,MM,78.96,overcast clouds,16.7619,96.5278,
33,Hilo,US,80.33,broken clouds,19.7297,-155.0900,
37,Lucea,JM,80.46,overcast clouds,18.4510,-78.1736,


In [9]:
len(hotel_df)

164

In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,}

# 6b. Iterate through the hotel DataFrame.
print('Collecting Hotel......')  
for index, row in hotel_df.iterrows():
         
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
   
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError): print("Hotel not found... skipping.")  
        
#Indicate the Data Loading is complete.
print("---------------------------")
print("Data Retrieval Complete    ")
print("---------------------------")

       

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
---------------------------
Data Retrieval Complete    
---------------------------


In [11]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,San Policarpo,PH,83.30,overcast clouds,12.1791,125.5072,Agapita M. Nicart
2,Butaritari,KI,81.39,broken clouds,3.0707,172.7902,Isles Sunset Lodge
4,Pochutla,MX,81.82,overcast clouds,15.7432,-96.4661,Hotel Posada San Jose
11,Padang,ID,75.25,clear sky,-0.9492,100.3543,Mercure Padang
20,Poum,NC,81.16,scattered clouds,-20.2333,164.0167,
27,Bengkulu,ID,76.66,broken clouds,-3.8004,102.2655,Grage Hotel Bengkulu
31,Hithadhoo,MV,82.40,clear sky,-0.6000,73.0833,Scoop Guest House
32,Thongwa,MM,78.96,overcast clouds,16.7619,96.5278,Mhaw Wun Marlar Park
33,Hilo,US,80.33,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
37,Lucea,JM,80.46,overcast clouds,18.4510,-78.1736,Grand Palladium Jamaica Resort & Spa


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df= hotel_df[hotel_df["Hotel Name"] != ""]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,San Policarpo,PH,83.30,overcast clouds,12.1791,125.5072,Agapita M. Nicart
2,Butaritari,KI,81.39,broken clouds,3.0707,172.7902,Isles Sunset Lodge
4,Pochutla,MX,81.82,overcast clouds,15.7432,-96.4661,Hotel Posada San Jose
11,Padang,ID,75.25,clear sky,-0.9492,100.3543,Mercure Padang
27,Bengkulu,ID,76.66,broken clouds,-3.8004,102.2655,Grage Hotel Bengkulu
31,Hithadhoo,MV,82.40,clear sky,-0.6000,73.0833,Scoop Guest House
32,Thongwa,MM,78.96,overcast clouds,16.7619,96.5278,Mhaw Wun Marlar Park
33,Hilo,US,80.33,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
37,Lucea,JM,80.46,overcast clouds,18.4510,-78.1736,Grand Palladium Jamaica Resort & Spa
39,Saint-Philippe,RE,75.22,clear sky,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))